In [25]:
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import os
import chromadb

In [ ]:
import sys
!{sys.executable} -m pip install sentence-transformers

In [31]:
#docs: cours sur l'agent
pdf_dir = "Docs"

all_documents = []


splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    separators=["\n\n", "\n", " ", ""]
)

for filename in os.listdir(pdf_dir):
    if filename.endswith(".pdf"):
        path = os.path.join(pdf_dir, filename)
        loader = PyPDFLoader(path)
        pages = loader.load()

        for page in pages:
            page.metadata["source"] = filename

        chunks = splitter.split_documents(pages)
        all_documents.extend(chunks)


In [9]:
all_documents

Document(metadata={'source': 'Chapitre I.pdf', 'page': 2}, page_content='L’Intelligence Artificielle \n« L’objectif de l’IA est de doter les systèmes informatiques/machines de capacités \nintellectuelles comparables à celles des êtres humains »\n• À l’époque, les ordinateurs n’avaient pas de capacités de calculs, il y avait pas de cloud\ncomputing, pas de big data…\n• Avant, l’IA était présente au niveau de la recherche scientifique seulement.\n• Aujourd’hui, nous pouvons avoir des ordinateurs ultra rapide, calcul parallèle, grâce à')

In [29]:
from langchain.embeddings import HuggingFaceEmbeddings

embedding_fn = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [32]:

vectorstore = Chroma(
    persist_directory="./chromadb",
    embedding_function=embedding_fn
)

vectorstore.add_documents(all_documents)
vectorstore.persist()


D:\Users\hp\AppData\Local\Temp\ipykernel_1636\1547584207.py:7: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [33]:
retriever = vectorstore.as_retriever()

def rag_query(query: str):
    results = retriever.get_relevant_documents(query)
    return "\n\n".join([
        f"Source: {doc.metadata.get('source', 'inconnu')}\n{doc.page_content}"
        for doc in results
    ])


In [44]:
rag_query("qu'est-ce que JADE")

'Source: Chapitre V.pdf\nL’architecture Interne du JADE Runtime\n\nSource: Chapitre V.pdf\nAgent JADE\n• Un agent JADE est une classe qui hérite de la classe Agent et qui redéfinie des méthodes\nidentifiant le cycle de vie de l’agent sur la plateforme JADE.\n• La méthode setup est la première méthode qui sera appelée après instanciation de l’agent\npar le container.\n• La méthode doDelete permet de demander au container de détruire l’agent.\n• Avant que l’agent soit détruit, la méthode takedown est appelée.\n• Un agent peut se déplacer d’un container à l’autre.\n\nSource: Chapitre I.pdf\nces ressources, l’IA est devenu une réalité.\n\nSource: Chapitre V.pdf\ndes agents.\nChaque instance du JADE est appelée conteneur « Container: instance de JADE », et peut\ncontenir/déployer plusieurs agents.\nUn ensemble de conteneurs(peuvent être démarré dans plusieurs machine) constituent une\nplateforme.'